In [71]:
from rdkit.Chem import MACCSkeys                       #MACCS Keys
from rdkit import Chem, DataStructs
import numpy as np
import pandas as pd
import os 
import xlrd 
import openpyxl

def fingerprint_bitgenerator(fps):
    np_fps = []
    
    for fp in fps:
        arr = np.zeros((1,))
        DataStructs.ConvertToNumpyArray(fp,arr)
        np_fps.append(arr)
        
    return np_fps

wdir = 'C:/jupyter_devel/kist-europe/QSAR/mmc1_data/'
xlsx = '1-s2.0-S0147651314004047-mmc1.xls'
csv = 'mmc1-androgen.csv'

### - to save data as csv which we can use

In [ ]:
#Data Load 
wb = xlrd.open_workbook(wdir+xlsx)
ws = wb.sheet_by_index(2) # index(1) >> androgen data  |  index(2) >> estrogen data  
nrows = sheet.nrows

#Get data as list type
row_val = []
for row_num in range(nrows):
    row_val.append(ws.row_values(row_num))
    
#To save as csv file 
df_to_save = pd.DataFrame(row_val)
#df_to_save.to_csv(wdir+'mmc1-estrogen.csv')

### - data preprocess

In [76]:
df = pd.read_csv(wdir+csv)

smi = df['SMILES']
sd = [Chem.MolFromSmiles(m) for m in smi]

maccfps = [MACCSkeys.GenMACCSKeys(m) for m in sd]
maccfps_bit = fingerprint_bitgenerator(maccfps)
fp_df = pd.DataFrame(maccfps_bit,columns=list(range(len(maccfps_bit[0]))))

df_id = df[['Set','Class']]
df_label_fp = pd.concat([df_id,fp_df],axis=1)

df_colnames = df_label_fp.columns[2:]

#Remove features with low variance
remove_cols = []
for c in df_colnames:
    if df_label_fp[c].std() < 0.01:
        remove_cols.append(c)
        
df_label_fp = df_label_fp.drop(remove_cols, axis=1)
corr = df_label_fp.corr().iloc[0]

df_label_fp.to_csv(wdir+'mmc1-androgen-maccs-activity.csv',index=False)


ArgumentError: Python argument types in
    rdkit.Chem.rdMolDescriptors.GetMACCSKeysFingerprint(NoneType)
did not match C++ signature:
    GetMACCSKeysFingerprint(class RDKit::ROMol mol)

 - invalid SMILES string, which results in invalid molecule object (from SMILES using MolFromSmiles method of RDKit), but I tried excluding such molecules by having an exception (even though these are drawn from a popular dataset).
 
 
 - You are passing a None where a molecule is expected. Note that unparsable molecules normally return None on error, so you should check for this and not process them further.
 
- The SMILES strings are not correct, rings should be enumerated. In the first
string for example, all four rings are labelled with '1' but it should look
like this: c1ccc(c(c1)C2CC(=NN2c3ccccc3F)c4ccc(cc4)Br)Cl. That is why RDKit
is throwing an error. Which software did you use to create the SMILES

- https://github.com/rdkit/rdkit/issues/866

- https://sourceforge.net/p/rdkit/mailman/message/24007265/